In [1]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex

import import_ipynb

import data_utils as data


tt = gp.Model('IIITB Exams Timetable')

1002
[(55, 57), (7, 17), (26, 30), (7, 26), (7, 35), (48, 54), (21, 37), (6, 57), (10, 27), (41, 42), (10, 36), (25, 43), (3, 6), (44, 56), (3, 24), (14, 24), (14, 33), (14, 51), (15, 16), (47, 48), (17, 44), (18, 21), (29, 36), (48, 49), (21, 32), (6, 43), (29, 45), (2, 27), (25, 29), (2, 36), (2, 45), (22, 32), (13, 51), (3, 28), (16, 56), (55, 56), (7, 16), (5, 46), (5, 55), (17, 57), (6, 29), (29, 31), (6, 38), (6, 56), (33, 37), (10, 35), (10, 44), (25, 42), (51, 54), (14, 32), (5, 41), (28, 43), (6, 24), (9, 57), (29, 35), (29, 44), (6, 42), (21, 40), (39, 48), (20, 53), (2, 26), (13, 32), (43, 45), (24, 41), (35, 41), (32, 54), (13, 50), (16, 46), (16, 55), (28, 29), (5, 27), (9, 25), (5, 36), (47, 51), (5, 45), (6, 10), (17, 56), (29, 30), (6, 46), (2, 3), (10, 25), (24, 27), (32, 40), (3, 4), (24, 36), (35, 36), (35, 45), (24, 45), (13, 54), (9, 29), (6, 14), (6, 23), (42, 44), (39, 47), (2, 7), (12, 39), (20, 52), (12, 48), (4, 44), (23, 57), (13, 22), (24, 31), (43, 44), (13

## Make one binary variable for each room-course-timeslot-day combination
$$
E_{crtd} =
\begin{cases} 
1 & \text{if a course $c$ exam is scheduled in room $r$ at time $t$ on day $d$,} \\
0 & \text{Otherwise.}
\end{cases}
$$

In [2]:
def exam_length(course:int):
    return 120

def exam_slots(course: int):
    # return (data.get_exam_duration(course)//15)

    return int(exam_length(course)//15)
#Dhruv's part:

# session_spans = {s: begin_end_pairs(session_length(s)) for s in range(n_sessions)}
# print(data.begin_end_pairs(exam_slots(1)))
exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c,r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=230,
    name="C",
)



## Hard Constraints

In [3]:
def signum(x):
    if x>0:
        return 1
    return 0

def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if  data.overlap(b,e,t)]

In [4]:
# Exactly one slot for each course
# sum of enrolments in courses assigned to a room <= room capacity
# no clash for any student
# all slots for a course on any day are contiguous
# exam lengths for each course

In [5]:


# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)

tt.update()

In [6]:
# Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
                          for c in range(data.n_courses)
                          for b,e in covering_sessions(c, t)) <= data.get_room_capacity(r)
              for d in range(data.n_days)
              for r in range(data.n_rooms)
              for t in range(data.n_times)),
              name = "room_capacity_constraint"
             )

# P = tt.addVars(
#     [
#         (c, r, b, e, d)
#         for c in range(data.n_courses)
#         for r in range(data.n_rooms)
#         for b, e in exam_spans[c]
#         for d in range(data.n_days)
#     ],
#     vtype=GRB.INTEGER,
#     name = 'P',
#     lb = 0, 
#     ub = 230,
# )


# for c,r,b,e,d in P:
#     room_capacity = data.get_room_capacity(r)
#     tt.addConstr(P[c,r,b,e,d] >=0, name = 'P1_constraint')
#     tt.addConstr(P[c,r,b,e,d] <= room_capacity*E[c,b,e,d], name = 'P2_constraint')
#     tt.addConstr(P[c,r,b,e,d] <= C[c,r]*(1-E[c,b,e,d]), name = 'P3_constraint')
#     tt.addConstr(P[c,r,b,e,d] >= (C[c,r] - room_capacity)*(1-E[c,b,e,d]), name = 'P4_constraint')


# tt.update()

# tt.addConstrs(
#     (gp.quicksum(P[c, r, b, e, d] 
#                  for r in range(data.n_rooms) 
#                  for b, e in covering_sessions(c, t)) <= data.get_room_capacity(r)
#      for d in range(data.n_days) 
#      for c in range(data.n_courses) 
#      for t in range(data.n_times)),
#     name='room_capacity_constraint'
# )

tt.update()


In [7]:
# Constraint 3: All students for a course should write the exam 

# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                           for r in range(data.n_rooms)
#                           for b,e in covering_sessions(c, t)) <= data.get_course_strength(c)
#               for d in range(data.n_days)
#               for c in range(data.n_courses)
#               for t in range(data.n_times)),
#               name = 'course_enrollment_constraint'
#              )

# # Constraint 3.1: All students for a course should write the exam
# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                            for d in range(data.n_days)
#                           for r in range(data.n_rooms)
#                           for t in range(data.n_times)
#                           for b,e in covering_sessions(c, t)) == data.get_course_strength(c)
#               for c in range(data.n_courses)),
#               name = 'course_enrollment_constraint'
#              )

# Constraint 3.1: All students for a course should write the exam

tt.addConstrs(
    gp.quicksum(C[c,r] for r in range(data.n_rooms)) == data.get_course_strength(c)
    for c in range(data.n_courses)
)



# tt.addConstrs(
#     (gp.quicksum(P[c, r, b, e, d] 
#                  for r in range(data.n_rooms) 
#                  for b, e in covering_sessions(c, t)) == data.get_course_strength(c)
#      for d in range(data.n_days) 
#      for c in range(data.n_courses) 
#      for t in range(data.n_times)),
#     name='course_enrollment_constraint'
# )

tt.update()


In [8]:
# # Constraint 4: Student clash constraint

# # for each student, for each day, for each time slot, at most one exam
# # sigma Ecbed <= 1 for courses enrolled by student s


# tt.addConstrs((gp.quicksum(E[c,b,e,d] for c in data.get_courses_student(s) 
#                 for b,e in covering_sessions(c, t)) <= 1
#                 for s in range(data.n_students)
#                 for d in range(data.n_days)
#                 for t in range(data.n_times)),
#               name = 'student_clash_constraint'
#              )

tt.addConstrs((gp.quicksum(E[c, b, e, d] for c in data.intersection_pairs[i] for b, e in covering_sessions(c, t)) <= 1
               for i in range(data.n_intersection_pairs)
               for d in range(data.n_days)
               for t in range(data.n_times)),
              name='student_clash_constraint'
             )

tt.update()

In [9]:
# Constraint 5: No course can have more than 3 rooms alloted


M = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'M',
)

D1 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D1',
)

D2 = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        for r in range(data.n_rooms)
    ],
    vtype=GRB.BINARY,
    name = 'D2',
)



for c,r in M:
    room_capacity = data.get_room_capacity(r)
    tt.addConstr(M[c,r] <=1, name = 'M1_constraint')
    tt.addConstr(M[c,r] <=C[c,r], name = 'M2_constraint')
    tt.addConstr(M[c,r] >=D1[c,r], name = 'M3_constraint')
    tt.addConstr(M[c,r] >=C[c,r]-(room_capacity*(1-D2[c,r])), name = 'M4_constraint')
    tt.addConstr(D1[c,r]+D2[c,r] == 1, name = 'M5_constraint')

# tt.addConstrs(
#     (
#         # gp.quicksum(signum(C[c, r]) for r in range(data.n_rooms)) <= 3
#         gp.quicksum(min(1, C[c, r]) for r in range(data.n_rooms)) <= 3
#         for c in range(data.n_courses)
#     ),
#     name="course_room_constraint",
# )

tt.addConstrs(
    (
        gp.quicksum(M[c, r] for r in range(data.n_rooms)) <= 3
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)

tt.update()

In [10]:
# Constraint 6: Lab exams are conducted in labs
# check_cse_lab_room and check_ece_lab_room 

tt.addConstrs(
    (
        gp.quicksum(C[c, r] for r in (data.get_not_rooms_course(c)) )== 0
        for c in range(data.n_courses)
    ),
    name="lab_exam_constraint",
)

tt.update()

Course: 0
Rooms: [4, 5, 18]
Course: 1
Rooms: [4, 5, 18]
Course: 2
Rooms: [4, 5, 18]
Course: 3
Rooms: [4, 5, 18]
Course: 4
Rooms: [4, 5, 18]
Course: 5
Rooms: [4, 5, 18]
Course: 6
Rooms: [4, 5, 18]
Course: 7
Rooms: [4, 5, 18]
Course: 8
Rooms: [4, 5, 18]
Course: 9
Rooms: [4, 5, 18]
Course: 10
Rooms: [4, 5, 18]
Course: 11
Rooms: [4, 5, 18]
Course: 12
Rooms: [4, 5, 18]
Course: 13
Rooms: [4, 5, 18]
Course: 14
Rooms: [4, 5, 18]
Course: 15
Rooms: [4, 5, 18]
Course: 16
Rooms: [4, 5, 18]
Course: 17
Rooms: [4, 5, 18]
Course: 18
Rooms: [4, 5, 18]
Course: 19
Rooms: [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Course: 20
Rooms: [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Course: 21
Rooms: [4, 5, 18]
Course: 22
Rooms: [4, 5, 18]
Course: 23
Rooms: [4, 5, 18]
Course: 24
Rooms: [4, 5, 18]
Course: 25
Rooms: [4, 5, 18]
Course: 26
Rooms: [4, 5, 18]
Course: 27
Rooms: [4, 5, 18]
Co

## Soft Constraints

In [11]:
# 1. No 2 exams on a day for a student

# S1 = gp.quicksum(
#     (gp.quicksum(E[c,b,e,d] for c in (data.get_courses_student(s))
#     for b,e in exam_spans[c])) >=2
#     for d in range(data.n_days)
#     for s in range(data.n_students)
# )
 # count the number of >=2 for each student, each day and return it into S1

S1 = tt.addVars(
    [
        (s, d)
        for s in range(data.n_students)
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name = 'S1',
)


for s,d in S1:
    tt.addConstr(
        gp.quicksum(E[c,b,e,d] for c in (data.get_courses_student(s))
        for b,e in exam_spans[c]) +  S1[s, d]*100 >=2
    )
    

tt.update()


## Objective Function

In [12]:
objective = S1.sum()
tt.setObjective(objective, sense=GRB.MINIMIZE)
tt.update()

In [13]:
# Maximize Gaps between exams for a student / batch
# tt.optimize()

# def early_stop_callback(model, where):
#     if where == GRB.Callback.MIP:
#         runtime = model.cbGet(GRB.Callback.RUNTIME)
#         mipgap = model.cbGet(GRB.Callback.MIPGAP)
#         print("runtime:", runtime)
#         print("mipgap:", mipgap)
#         if runtime > 120 and mipgap > 0.15:
#             print("Early stopping triggered: runtime > 120 sec and MIPGap > 15%")
#             model.terminate()

# # Optimize with the early stopping callback
# tt.optimize(early_stop_callback)

import time


def cb(model, where):
    if where == GRB.Callback.MIPNODE:
        # Get model objective
        obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

        # Has objective changed?
        if abs(obj - model._cur_obj) > 1e-8:
            # If so, update incumbent and time
            model._cur_obj = obj
            model._time = time.time()

    # Terminate if objective has not improved in 20s
    if time.time() - model._time > 60:
        model.terminate()

tt._cur_obj = float('inf')
tt._time = time.time()

tt.optimize(callback=cb)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2615578 - for non-commercial use only - registered to sa___@iiitb.ac.in
Optimize a model with 90992 rows, 18582 columns and 1402248 nonzeros
Model fingerprint: 0x4da477bc
Model has 5890 quadratic constraints
Variable types: 0 continuous, 18582 integer (16784 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 3e+02]
  QRHS range       [6e+00, 2e+02]
Presolve removed 84644 rows and 8122 columns
Presolve time: 3.30s
Presolved: 173088 rows, 171500 columns, 2179314 nonzeros
Variable types: 0 continuous, 171500 integer (8996 binary)
Deterministic concurrent LP optimizer: primal 

In [14]:
E_sol = tt.getAttr('x', E)
C_sol = tt.getAttr('x', C)
M_sol = tt.getAttr('x', M)
D1_sol = tt.getAttr('x', D1)
D2_sol = tt.getAttr('x', D2)

#Get all the values from E_sol whose value is 1
solution=[]
for i in E_sol:
    if E_sol[i] == 1:
        solution.append(i)

# for i in solution:
#     print(i)

In [15]:
# print(C_sol)
solution_rooms=[]
for i in C_sol:
    if C_sol[i] > 0:
        solution_rooms.append(i)

# for i in solution_rooms:
#     print(i)

In [16]:
! pip install tabulate
! pip install matplotlib

In [17]:
for c,b,e,d in solution:
    l = []
    for(r) in range(data.n_rooms):
        if C_sol[c,r] > 0:
            l.append((data.get_room_num(r), C_sol[c,r]))
    print(f"Course {data.course_id_code[c]} has {data.get_course_strength(c)} and is scheduled in slot {data.times[b]} to {data.times[e]} on day {data.days[d]} in room {l}")

Course AIM 101  has 59 and is scheduled in slot 11:00 to 13:00 on day Wed in room [('A226', 12.0), ('Ramanujan Basement 02 (B- Block)', 41.0), ('R-310', 6.0)]
Course AIM 102  has 59 and is scheduled in slot 08:30 to 10:30 on day Mon in room [('Ramanujan Basement 01 (A- Block)', 58.0), ('Ramanujan Basement 02 (B- Block)', 1.0)]
Course AIM 704  has 20 and is scheduled in slot 10:45 to 12:45 on day Tue in room [('A304/A305', 20.0)]
Course AIM 821  has 21 and is scheduled in slot 10:45 to 12:45 on day Fri in room [('A303', 21.0)]
Course AIM 825  has 158 and is scheduled in slot 15:15 to 17:15 on day Fri in room [('R102', 60.0), ('R203', 95.0), ('R101', 3.0)]
Course AIM 829  has 132 and is scheduled in slot 11:00 to 13:00 on day Thu in room [('R308', 15.0), ('Ramanujan Basement 02 (B- Block)', 80.0), ('Ramanujan Basement 04 (D- Block)', 37.0)]
Course AIM 831  has 175 and is scheduled in slot 11:00 to 13:00 on day Mon in room [('R102', 60.0), ('A106', 25.0), ('A304/A305', 90.0)]
Course AIM 8

In [18]:
from collections import defaultdict
from tabulate import tabulate

schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in range(data.n_rooms):
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri"]

print("EXAM TIMETABLE")
print("=" * 50)
for day in weekdays:
    if day in schedule_by_day:
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        for entry in sorted_entries:
            del entry["start"]
        print(f"\n{day}")
        print("-" * 50)
        print(tabulate(sorted_entries, headers="keys", tablefmt="grid"))

EXAM TIMETABLE

Mon
--------------------------------------------------
+-------------+------------+---------------+-----------------------------------------------------------------------------------------------------+
| Course      |   Strength | Time          | Rooms                                                                                               |
+=============+============+===============+=====================================================================================================+
| CSE 754     |         25 | 08:00 - 10:00 | R305 (Capacity: 1.0), A303 (Capacity: 24.0)                                                         |
+-------------+------------+---------------+-----------------------------------------------------------------------------------------------------+
| AIM 846     |         26 | 08:00 - 10:00 | A307 (Capacity: 25.0), A308 (Capacity: 1.0)                                                         |
+-------------+------------+---------------+---

In [19]:
# ## Verification of the solution

# # 1 - No student should have two exams at the same time

# def time_to_minutes(time_str):
#     """Converts a HH:MM string to minutes since midnight."""
#     hh, mm = map(int, time_str.split(":"))
#     return hh * 60 + mm

# # Build a dictionary that maps each course (c) to its one exam slot details.
# # Assume each course appears only once in the solution.
# course_slots = {}
# for c, b, e, d in solution:
#     # Save day and exam start/end times (converted to minutes) for each course
#     day = data.days[d]
#     start = time_to_minutes(data.times[b])
#     end   = time_to_minutes(data.times[e])
#     course_slots[c] = (day, start, end)

# # Check for any student conflicts between every pair of distinct courses.
# clash_found = False
# course_indices = list(course_slots.keys())
# for i in range(len(course_indices)):
#     c1 = course_indices[i]
#     day1, start1, end1 = course_slots[c1]
#     for j in range(i + 1, len(course_indices)):
#         c2 = course_indices[j]
#         day2, start2, end2 = course_slots[c2]
#         # Check if courses are on the same day and if their times overlap:
#         if day1 == day2 and (start1 < end2 and start2 < end1):
#             # Check intersection of enrolled students; get_intersection_of_courses returns set of common student IDs.
#             common_students = set(data.course_student[c1]) & set(data.course_student[c2])
#             if common_students:
#                 clash_found = True
#                 print(f"Clash found between Course {data.course_id_code[c1]} and Course {data.course_id_code[c2]}")
#                 print("Common student(s):", common_students)
#                 print(f"--> {data.course_id_code[c1]}: {data.times[b]} - {data.times[e]} ;",  # later you can format times properly
#                       f"{data.course_id_code[c2]}: {data.times[b]} - {data.times[e]}\n")

# if not clash_found:
#     print("No student clashes found!")


# # 2 - The total number of students writing an exam in a room should not exceed the room capacity


# room_violations = False

# # Process each room separately
# for r in range(data.n_rooms):
#     # Build a schedule per day for room r.
#     room_schedule = defaultdict(list)
#     for c, b, e, d in solution:
#         allocated = C_sol[c, r]
#         if allocated > 0:
#             day = data.days[d]
#             start = time_to_minutes(data.times[b])
#             end = time_to_minutes(data.times[e])
#             course_code = data.course_id_code[c]
#             room_schedule[day].append((start, end, allocated, course_code))
    
#     # For each day, process the exam intervals for room r.
#     for day, intervals in room_schedule.items():
#         events = []
#         for start, end, alloc, course in intervals:
#             # When exam starts, add the number of students;
#             # when exam ends, subtract that number.
#             events.append((start, alloc))
#             events.append((end, -alloc))
#         events.sort(key=lambda x: x[0])
        
#         current_alloc = 0
#         for time, change in events:
#             current_alloc += change
#             if current_alloc > data.get_room_capacity(r):
#                 room_violations = True
#                 print(f"Room capacity violation on {day} in room {data.get_room_num(r)}:")
#                 print(f"   Room capacity: {data.room_capacity[r]}, allocated: {current_alloc}")
#                 break  # Report once per day per room

# if not room_violations:
#     print("No room capacity violations found!")


In [20]:
for s in range (data.n_students):
    l=[]
    course_list=data.get_courses_student(s)
    for c,b,e,d in solution:
        if E_sol[c,b,e,d] == 1 and c in course_list:
            l.append((data.course_id_code[c], data.times[b], data.times[e], data.days[d]))
    
    print(f"Student {s+1} has exams for courses {l}")

Student 1 has exams for courses [('AIM 101 ', '11:00', '13:00', 'Wed'), ('AIM 102 ', '08:30', '10:30', 'Mon'), ('CSE 102 ', '14:00', '16:00', 'Fri'), ('CSE 102 P -B', '08:00', '10:00', 'Thu'), ('CSE 104 ', '08:00', '10:00', 'Fri'), ('DAS 101 ', '11:00', '13:00', 'Tue'), ('DAS 101P -B', '11:00', '13:00', 'Mon'), ('DHS 101B ', '14:45', '16:45', 'Wed'), ('DHS 201 ', '11:00', '13:00', 'Thu')]
Student 2 has exams for courses [('AIM 101 ', '11:00', '13:00', 'Wed'), ('AIM 102 ', '08:30', '10:30', 'Mon'), ('CSE 102 ', '14:00', '16:00', 'Fri'), ('CSE 102 P -B', '08:00', '10:00', 'Thu'), ('CSE 104 ', '08:00', '10:00', 'Fri'), ('DAS 101 ', '11:00', '13:00', 'Tue'), ('DAS 101P -B', '11:00', '13:00', 'Mon'), ('DHS 101B ', '14:45', '16:45', 'Wed'), ('DHS 201 ', '11:00', '13:00', 'Thu')]
Student 3 has exams for courses [('AIM 101 ', '11:00', '13:00', 'Wed'), ('AIM 102 ', '08:30', '10:30', 'Mon'), ('CSE 102 ', '14:00', '16:00', 'Fri'), ('CSE 102 P -B', '08:00', '10:00', 'Thu'), ('CSE 104 ', '08:00', '